In [107]:
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(open('big.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)


In [92]:
correct('Speling')

'spelling'

In [109]:
NWORDS = train(words(open('big.txt').read()))

print(correct('Thew'))
print(correct('Korrecter'))
print(correct('reciet'))
print(correct('rember'))

whew
corrected
recite
member


In [101]:
from collections import Counter

NWORDS = train(words(open('big.txt').read()))
WORDS = Counter(words(open('big.txt').read()))

def correction(word):
    return correct(word)

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def unit_tests():
    assert correction('speling') == 'spelling'              # insert
    assert correction('korrectud') == 'corrected'           # replace 2
    assert correction('bycycle') == 'bicycle'               # replace
    assert correction('inconvient') == 'inconvenient'       # insert 2
    assert correction('arrainged') == 'arranged'            # delete
    assert correction('peotry') =='poetry'                  # transpose
    assert correction('peotryy') =='poetry'                 # transpose + delete
    assert correction('word') == 'word'                     # known
    assert correction('quintessential') == 'quintessential' # unknown
    assert words('This is a TEST.') == ['this', 'is', 'a', 'test']
#    print(Counter(words('This is a test. 123; A TEST this is.')))
    assert Counter(words('This is a test. 123; A TEST this is.')) == (
           Counter({'a': 2, 'is': 2, 'test': 2, 'this': 2}))
#    print(len(WORDS))
    #assert len(WORDS) == 32192
    #assert sum(WORDS.values()) == 1115504
#    assert WORDS.most_common(10) == [
#     ('the', 79808),
#     ('of', 40024),
#     ('and', 38311),
#     ('to', 28765),
#     ('in', 22020),
#     ('a', 21124),
#     ('that', 12512),
#     ('he', 12401),
#     ('was', 11410),
#     ('it', 10681)]
#    assert WORDS['the'] == 79808
    print(P('quintessential'))
#    assert P('quintessential') == 0
    assert 0.07 < P('the') < 0.1
    return 'unit_tests pass'

print(unit_tests())

0.0
unit_tests pass


In [102]:
def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.clock()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.clock() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

spelltest(Testset(open('spell-testset1.txt'))) # Development set
#spelltest(Testset(open('spell-testset2.txt'))) # Final test set

75% of 270 correct (6% unknown) at 33 words per second 
